
**Quantitative Momentum Strategy**

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.
Library Import














In [4]:
!sudo pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter 
import math 
from scipy import stats 


In [6]:
stocks = pd.read_csv('/content/sp_500_stocks.csv')
stocks= stocks.drop(labels=[135,219,467,484],axis=0)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


**Acquiring an API Token**

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a *secrets.py *file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.


In [7]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'


**Making Our First API Call**

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.


In [8]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avg10Volume': 77211985,
 'avg30Volume': 76562462,
 'beta': 1.2608701011983872,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 162.52,
 'day30ChangePercent': 0.16961922916250666,
 'day50MovingAvg': 146.21,
 'day5ChangePercent': 0.05359719557811577,
 'dividendYield': 0.006024928331950899,
 'employees': 148043,
 'exDividendDate': '2022-04-26',
 'float': 0,
 'marketcap': 2520206266756,
 'maxChangePercent': 60.362827767164816,
 'month1ChangePercent': 0.1650480821679857,
 'month3ChangePercent': -0.0853232553614053,
 'month6ChangePercent': -0.06835131459191636,
 'nextDividendDate': '',
 'nextEarningsDate': '2022-07-19',
 'peRatio': 25.1259392540525,
 'sharesOutstanding': 16470525103,
 'ttmDividendRate': 0.8901847715258798,
 'ttmEPS': 6.42,
 'week52change': 0.05933556799580005,
 'week52high': 183.8,
 'week52highSplitAdjustOnly': 185.97,
 'week52low': 131.31,
 'week52lowSplitAdjustOnly': 132.47,
 'year1ChangePercent': 0.0540208296943931,
 'year2ChangePercent': 0.5791596094429502,
 'year5Cha


**Parsing Our API Call**

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.


In [9]:
data['year1ChangePercent']

0.0540208296943931


**Executing A Batch API Call & Building Our DataFrame**

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

 More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.


In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        

In [11]:
 symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year_Price_Return', 'Number_of_Shares_to_Buy']


A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS
GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INF

In [12]:
for symbol_string in symbol_strings:
  print(symbol_string)

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS
GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INF

In [13]:
 batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
 data = requests.get(batch_api_call_url).json()
 print(data)

{'ZTS': {'stats': {'companyName': 'Zoetis Inc', 'marketcap': 85732726817, 'week52high': 255.37, 'week52low': 160.3, 'week52highSplitAdjustOnly': 253.06, 'week52lowSplitAdjustOnly': 160.88, 'week52change': -0.10547503847936973, 'sharesOutstanding': 477672733, 'float': 0, 'avg10Volume': 1995488, 'avg30Volume': 1898054, 'day200MovingAvg': 187.93, 'day50MovingAvg': 173.3, 'employees': 11411, 'ttmEPS': 4.38, 'ttmDividendRate': 1.27199675592537, 'dividendYield': 0.006969143836883161, 'nextDividendDate': '', 'exDividendDate': '2022-07-07', 'nextEarningsDate': '2022-07-23', 'peRatio': 40.73365811204865, 'beta': 0.9158729049710679, 'maxChangePercent': 5.216651865344247, 'year5ChangePercent': 1.932770705722729, 'year2ChangePercent': 0.22505507399948219, 'year1ChangePercent': -0.118152610827588, 'ytdChangePercent': -0.2747976242896664, 'month6ChangePercent': -0.1287705754374153, 'month3ChangePercent': -0.05989199452578572, 'month1ChangePercent': 0.11584426659939563, 'day30ChangePercent': 0.114646

In [14]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']


 we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [15]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,130.51,-0.198966,N/A
1,AAL,14.73,-0.261323,N/A
2,AAP,196.25,-0.060372,N/A
3,AAPL,157.70,0.054733,N/A
4,ABBV,150.68,0.335121,N/A
...,...,...,...,...
496,YUM,120.26,0.05932,N/A
497,ZBH,112.10,-0.304455,N/A
498,ZBRA,335.75,-0.37879,N/A
499,ZION,54.90,0.061604,N/A



**Removing Low-Momentum Stocks**

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [16]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.49,2.316478,N/A
1,OXY,65.00,1.542354,N/A
2,DVN,57.40,1.433149,N/A
3,MRO,22.28,1.083148,N/A
4,APA,33.78,1.041966,N/A
5,CF,86.45,0.867995,N/A
6,VLO,107.43,0.86313,N/A
7,MPC,90.51,0.788241,N/A
8,DLTR,172.51,0.770249,N/A
9,COP,92.58,0.73083,N/A



**Calculating the Number of Shares to Buy**

 we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Notebook.



In [17]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)


Enter the value of your portfolio:100000
100000


In [18]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.49,2.316478,24
1,OXY,65.00,1.542354,30
2,DVN,57.40,1.433149,34
3,MRO,22.28,1.083148,89
4,APA,33.78,1.041966,59
5,CF,86.45,0.867995,23
6,VLO,107.43,0.86313,18
7,MPC,90.51,0.788241,22
8,DLTR,172.51,0.770249,11
9,COP,92.58,0.73083,21



Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

    High-quality momentum stocks show "slow and steady" outperformance over long periods of time
    Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

    1-month price returns
    3-month price returns
    6-month price returns
    1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality momentum.


In [19]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')


**Calculating Momentum Percentiles**

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

    One-Year Price Return
    Six-Month Price Return
    Three-Month Price Return
    One-Month Price Return

Here's how we'll do this:


In [20]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0





In [21]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
      
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100


In [22]:
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.245509
1      0.147705
2      0.500998
3      0.718563
4       0.94012
         ...   
496    0.730539
497    0.107784
498    0.051896
499    0.736527
500    0.391218
Name: One-Year Return Percentile, Length: 501, dtype: object
0      0.383234
1      0.489022
2      0.363273
3      0.572854
4      0.918164
         ...   
496    0.700599
497    0.433134
498    0.035928
499    0.287425
500    0.423154
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.656687
1      0.121756
2      0.333333
3      0.620758
4      0.736527
         ...   
496    0.770459
497    0.179641
498    0.219561
499    0.221557
500    0.702595
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.487026
1      0.958084
2       0.91018
3      0.936128
4      0.572854
         ...   
496    0.712575
497     0.39521
498    0.824351
499    0.209581
500    0.762475
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,128.19,N/A,-0.196633,0.245509,-0.143033,0.383234,-0.074891,0.656687,0.06856,0.487026,N/A
1,AAL,14.48,N/A,-0.268729,0.147705,-0.094926,0.489022,-0.225621,0.121756,0.177568,0.958084,N/A
2,AAP,196.75,N/A,-0.060817,0.500998,-0.148581,0.363273,-0.153707,0.333333,0.151978,0.91018,N/A
3,AAPL,161.08,N/A,0.053355,0.718563,-0.069745,0.572854,-0.084727,0.620758,0.163691,0.936128,N/A
4,ABBV,152.97,N/A,0.328292,0.94012,0.13168,0.918164,-0.050768,0.736527,0.081029,0.572854,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.16,N/A,0.059692,0.730539,-0.013355,0.700599,-0.035829,0.770459,0.103541,0.712575,N/A
497,ZBH,113.40,N/A,-0.302313,0.107784,-0.120813,0.433134,-0.200598,0.179641,0.056602,0.39521,N/A
498,ZBRA,348.72,N/A,-0.380022,0.051896,-0.358524,0.035928,-0.183775,0.219561,0.129089,0.824351,N/A
499,ZION,53.74,N/A,0.06156,0.736527,-0.172235,0.287425,-0.18327,0.221557,0.019374,0.209581,N/A



**Calculating the HQM Score**

We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.


In [23]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)



**Selecting the 50 Best Momentum Stocks**

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 50 entries.


In [24]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,128.19,N/A,-0.196633,0.245509,-0.143033,0.383234,-0.074891,0.656687,0.06856,0.487026,0.443114
1,AAL,14.48,N/A,-0.268729,0.147705,-0.094926,0.489022,-0.225621,0.121756,0.177568,0.958084,0.429142
2,AAP,196.75,N/A,-0.060817,0.500998,-0.148581,0.363273,-0.153707,0.333333,0.151978,0.91018,0.526946
3,AAPL,161.08,N/A,0.053355,0.718563,-0.069745,0.572854,-0.084727,0.620758,0.163691,0.936128,0.712076
4,ABBV,152.97,N/A,0.328292,0.94012,0.13168,0.918164,-0.050768,0.736527,0.081029,0.572854,0.791916
5,ABC,143.02,N/A,0.232511,0.902196,0.074051,0.864271,-0.145617,0.367265,0.005976,0.161677,0.573852
6,ABMD,285.45,N/A,-0.153323,0.315369,-0.089704,0.500998,-0.158771,0.307385,0.209102,0.98004,0.525948
7,ABT,115.13,N/A,-0.07705,0.477046,-0.130106,0.413174,-0.118563,0.487026,0.062812,0.443114,0.45509
8,ACN,300.73,N/A,-0.088407,0.451098,-0.173763,0.277445,-0.128059,0.43513,0.02653,0.231537,0.348802
9,ADBE,429.17,N/A,-0.350315,0.06986,-0.216416,0.205589,-0.063116,0.688623,0.115718,0.772455,0.434132



**Calculating the Number of Shares to buy**

We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.



In [25]:
portfolio_input()


Enter the value of your portfolio:1000000


In [26]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,128.19,152,-0.196633,0.245509,-0.143033,0.383234,-0.074891,0.656687,0.06856,0.487026,0.443114
1,AAL,14.48,1354,-0.268729,0.147705,-0.094926,0.489022,-0.225621,0.121756,0.177568,0.958084,0.429142
2,AAP,196.75,99,-0.060817,0.500998,-0.148581,0.363273,-0.153707,0.333333,0.151978,0.91018,0.526946
3,AAPL,161.08,121,0.053355,0.718563,-0.069745,0.572854,-0.084727,0.620758,0.163691,0.936128,0.712076
4,ABBV,152.97,128,0.328292,0.94012,0.13168,0.918164,-0.050768,0.736527,0.081029,0.572854,0.791916
5,ABC,143.02,137,0.232511,0.902196,0.074051,0.864271,-0.145617,0.367265,0.005976,0.161677,0.573852
6,ABMD,285.45,68,-0.153323,0.315369,-0.089704,0.500998,-0.158771,0.307385,0.209102,0.98004,0.525948
7,ABT,115.13,170,-0.07705,0.477046,-0.130106,0.413174,-0.118563,0.487026,0.062812,0.443114,0.45509
8,ACN,300.73,65,-0.088407,0.451098,-0.173763,0.277445,-0.128059,0.43513,0.02653,0.231537,0.348802
9,ADBE,429.17,45,-0.350315,0.06986,-0.216416,0.205589,-0.063116,0.688623,0.115718,0.772455,0.434132



**Formatting Our Excel Output**

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization.


In [27]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)



**Creating the Formats We'll Need For Our .xlsx File**

 We'll need four main formats for our Excel document:

    String format for tickers
    $XX.XX format for stock prices
    $XX,XXX format for market capitalization
    Integer format for the number of shares to purchase




In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [29]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)



**Saving Our Excel Output**

In [30]:
writer.save()